In [1]:
# read from json file
import json 
with open('data.jsonl') as jsonl_file:
    lines = jsonl_file.readlines()
jsonS = [json.loads(line) for line in lines]
print(len(jsonS))

3702


In [2]:
with open('annotations.jsonl') as jsonl_file:
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
print(annot[0])

{'text': 'DORNBIRN In der Schulgasse in Dornbirn hat eine 71,93 Quadratmeter große Wohnung für einen Quadratmeterpreis von 5533,71 Euro den Besitzer gewechselt. Dieser beinhaltet auch einen Pkw-Abstellplatz. Käufer der Wohnung mit 9,86 Quadratmetern Terrasse ist die ValLiLean Beteiligungs- und Immobilienverwaltungs GmbH. Beim Verkäufer handelt es sich um die Karrenblick Projekt GmbH.  Der Kaufpreis liegt bei 398.040 Euro. Unterzeichnet wurde der Kaufvertrag am 18. September. Die Verbücherung datiert mit Oktober 2020.', 'meta': {'source': 'vn', 'pattern': '19, 19, 96'}, '_input_hash': 1497722439, '_task_hash': 939340509, 'spans': [{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT'}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE'}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT'}, {'start': 48, 'end': 53, 'token_start': 8, 'to

In [3]:
a=[{'start':0, 'end':3, 'label': 'ORT'},{'start':5, 'end':8, 'label': 'ORT2'},{'start':10, 'end':13, 'label': 'ORT3'}]
b=3
def getLabel(listDict,idxStart):
    for i in range(0,len(listDict)):
        auxDict=listDict[i]
        a_s, a_e = auxDict['start'], auxDict['end']
        if idxStart>=a_s: # i.e. in any case stop when idxStart < a_s (as dict tokens are ordered ASC in list; i.e. by order they appear in text)
            if idxStart<=a_e: 
                return auxDict['label'] 
            elif i==len(listDict)-1: #case where idxStart > 'end' key of last dict in list
                if idxStart>a_e:
                    return 'O'
        else: return 'O' 
print(getLabel(a,b))

ORT


In [4]:
# now use above code and loop through all items of annot list:
for j in range(0,len(annot)):
    a = annot[j]
    # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
    b = a['spans']
    # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
    if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
        for i in range(0,len(b)): 
            b[i]['noWords']=b[i]['token_end']-b[i]['token_start']+1
        #print(b)

        # select dict of tokens
        c = a['tokens']
        # add label to each token
        for i in range(0,len(c)):
            c[i]['label'] = getLabel(b,c[i]['start'])
            # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
            if c[i]['label'] != "O":
                if i==0:
                    c[i]['label'] = "B-" + c[i]['label']
                else: 
                    if c[i]['label'] == c[i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                        c[i]['label'] = "I-" + c[i]['label']
                    else:
                        c[i]['label'] = "B-" + c[i]['label'] 
        annot[j]['tokens'] = c

In [5]:
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [6]:
print(train_sents[0])

[('DORNBIRN', 'B-ORT'), ('In', 'O'), ('der', 'O'), ('Schulgasse', 'B-STRASSE'), ('in', 'O'), ('Dornbirn', 'B-ORT'), ('hat', 'O'), ('eine', 'O'), ('71,93', 'B-FLAECHE'), ('Quadratmeter', 'O'), ('große', 'O'), ('Wohnung', 'B-IMMO_TYP'), ('für', 'O'), ('einen', 'O'), ('Quadratmeterpreis', 'O'), ('von', 'O'), ('5533,71', 'B-GESAMTPREIS'), ('Euro', 'O'), ('den', 'O'), ('Besitzer', 'O'), ('gewechselt', 'O'), ('.', 'O'), ('Dieser', 'O'), ('beinhaltet', 'O'), ('auch', 'O'), ('einen', 'O'), ('Pkw-Abstellplatz', 'O'), ('.', 'O'), ('Käufer', 'O'), ('der', 'O'), ('Wohnung', 'O'), ('mit', 'O'), ('9,86', 'B-TERRASSENGROESSE'), ('Quadratmetern', 'O'), ('Terrasse', 'O'), ('ist', 'O'), ('die', 'O'), ('ValLiLean', 'B-KAEUFER'), ('Beteiligungs-', 'I-KAEUFER'), ('und', 'I-KAEUFER'), ('Immobilienverwaltungs', 'I-KAEUFER'), ('GmbH.', 'I-KAEUFER'), ('Beim', 'O'), ('Verkäufer', 'O'), ('handelt', 'O'), ('es', 'O'), ('sich', 'O'), ('um', 'O'), ('die', 'O'), ('Karrenblick', 'B-VERKAEUFER'), ('Projekt', 'I-VERKAE

In [7]:
def word2features(sent, i):
    word = sent[i][0]
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag, # don't have PoS data
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

#def sent2tokens(sent):
#   # return [token for token, postag, label in sent]
#    return [token for token, label in sent]

In [8]:
sent2features(train_sents[0])[0]

{'bias': 1.0,
 'word.lower()': 'dornbirn',
 'word[-3:]': 'IRN',
 'word[-2:]': 'RN',
 'word.isupper()': True,
 'word.istitle()': False,
 'word.isdigit()': False,
 'BOS': True,
 '+1:word.lower()': 'in',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False}

In [9]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [10]:
print(y_train[0])

['B-ORT', 'O', 'O', 'B-STRASSE', 'O', 'B-ORT', 'O', 'O', 'B-FLAECHE', 'O', 'O', 'B-IMMO_TYP', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'B-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'I-KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-VERKAEUFER', 'I-VERKAEUFER', 'I-VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG']


In [11]:
print(len(X_train))
print(len(y_train))

140
140


In [12]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train[:120], y_train[:120])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs


/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [13]:
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,I-ORT,B-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,2.41,1.659,-1.68,1.875,-1.937,2.181,1.647,-1.059,1.981,-3.077,1.556,-2.038,0.844,-1.139,1.564,1.97,-2.25,0.87,1.67,-2.239
B-DATUM_VERBUECHERUNG,-1.445,-0.474,4.753,0.0,0.0,0.0,0.0,0.0,0.0,-0.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERBUECHERUNG,-0.529,0.0,1.172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,-0.758,0.0,0.0,-0.578,4.744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.244,0.0,0.0,0.0,3.982,0.0,0.0,0.0,0.0,0.0,0.0,-0.034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.129
B-FLAECHE,1.508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,0.0,0.0,0.0,-0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,1.407,0.0,-0.09,0.0,0.0,0.0,0.0,2.202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.147,0.0
I-GESAMTPREIS,0.548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,0.458,0.0,-0.168,0.0,-0.106,0.0,0.0,0.0,-0.213,3.242,0.0,-0.259,0.0,0.0,0.0,0.35,-0.0,0.0,0.0,-0.231
I-IMMO_TYP,0.057,0.0,0.0,0.0,0.0,-0.136,0.0,0.0,-0.029,3.934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.106,0.0,0.0


In [14]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORT',
 'B-STRASSE',
 'B-FLAECHE',
 'B-IMMO_TYP',
 'B-GESAMTPREIS',
 'B-TERRASSENGROESSE',
 'B-KAEUFER',
 'I-KAEUFER',
 'B-VERKAEUFER',
 'I-VERKAEUFER',
 'B-DATUM_VERTRAG',
 'I-DATUM_VERTRAG',
 'B-DATUM_VERBUECHERUNG',
 'I-DATUM_VERBUECHERUNG',
 'B-QMPREIS',
 'I-IMMO_TYP',
 'I-STRASSE',
 'I-GESAMTPREIS',
 'I-ORT']

In [15]:
X_test = X_train[120:]
y_test = y_train[120:]

In [16]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.9136936747309867

In [17]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

B-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
I-DATUM_VERBUECHERUNG      1.000     1.000     1.000        15
      B-DATUM_VERTRAG      0.933     0.933     0.933        15
      I-DATUM_VERTRAG      0.931     0.964     0.947        28
            B-FLAECHE      0.944     1.000     0.971        17
        B-GESAMTPREIS      1.000     1.000     1.000        13
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.944     0.773     0.850        22
           I-IMMO_TYP      0.000     0.000     0.000         1
            B-KAEUFER      0.833     0.833     0.833        12
            I-KAEUFER      0.773     0.944     0.850        18
                B-ORT      0.941     0.941     0.941        34
                I-ORT      0.000     0.000     0.000         1
            B-QMPREIS      1.000     1.000     1.000        11
            B-STRASSE      0.923     0.857     0.889  

/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'I-ORT', 'B-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthias/opt/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/